## Importing Libraries

In [1]:
#Elsevier Libraries
from elsapy.elsclient import ElsClient
from elsapy.elsprofile import ElsAuthor, ElsAffil
from elsapy.elsdoc import FullDoc, AbsDoc
from elsapy.elssearch import ElsSearch

#Libraries for URL access
import json
import csv
import pprint
import requests
import xmltodict
import urllib3

In [2]:
#xml.etree.ElementTree — The ElementTree XML API
import xml.etree.ElementTree as ET

In [3]:
#Gen Libraries
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()

import matplotlib.pyplot as plt
%matplotlib inline

In [148]:
info_list = [] #Creating an empty list to store the  info
#Rerunning this will delete the data that will be stored further
#Run only once

In [158]:
#Create a num array of 0 to 5900 spaces 100 apart per page. Each page of Search returns 100 entries until 6000 is reached
num = np.linspace(0, 5900, 60, dtype = int)
print(num)
#Maximum of 6000 searches are returned on calling the ElsSearch API

[   0  100  200  300  400  500  600  700  800  900 1000 1100 1200 1300
 1400 1500 1600 1700 1800 1900 2000 2100 2200 2300 2400 2500 2600 2700
 2800 2900 3000 3100 3200 3300 3400 3500 3600 3700 3800 3900 4000 4100
 4200 4300 4400 4500 4600 4700 4800 4900 5000 5100 5200 5300 5400 5500
 5600 5700 5800 5900]


## Accessing the Links - API Extraction

In [66]:
#config = {'apikey': '22d18e0c4fdc59bb8898844056b2be57'}
#Append "apikey" and "insttoken" as suggest in the ElsSearch document into a config file
config = {
    "apikey": "2dc442325fc67f2f275ec3157ef8df65",
 "insttoken": "6beb1f6c29d85f50029bf11c8de94d1b"
    }

client = ElsClient(config['apikey'])
client.inst_token = config['insttoken']

In [150]:
def XML_DOI(link):
    
    #defining a header's dictionary to pass through requests
    headers_dict = {"X-ELS-APIKey": "2dc442325fc67f2f275ec3157ef8df65", "X-ELS-Insttoken": "6beb1f6c29d85f50029bf11c8de94d1b", "Accept": "application/xml"}
    
    #x takes response of the HTTP request, passes link
    x = requests.get(link, headers=headers_dict)
    #print(x.text) #check
    
    #Save it as XML file
    with open("doi.xml", 'wb') as f:
        f.write(x.content)


#Below are check links to test the code
#https://api.elsevier.com/content/search/sciencedirect?start=0&count=100&query=hydrogen+production+using+alcohol+AND+pub-date+2021&apiKey=2dc442325fc67f2f275ec3157ef8df65&insttoken=6beb1f6c29d85f50029bf11c8de94d1b
#https://api.elsevier.com/content/search/sciencedirect?start=0&count=100&pub-date=2017&query=hydrogen+production+using+alcohol&apiKey=2dc442325fc67f2f275ec3157ef8df65&insttoken=6beb1f6c29d85f50029bf11c8de94d1b

In [151]:
link_list = []

#Loop to iterate over all papers
#UPI query can take many arguments.
#start, count, query are a few as name suggests
for i in range(0, 60):
    
    start = "https://api.elsevier.com/content/search/sciencedirect?" 
    count = "start=" + str(num[i]) + "&count=100"
    query = "&query=hydrogen+production+using+alcohol"
    endapi = "&apiKey=2dc442325fc67f2f275ec3157ef8df65&insttoken=6beb1f6c29d85f50029bf11c8de94d1b"
    
    link = start + count + query # + endapi 
    #print(link)
    link_list.append(link)
#subscription = "&subscribed=true"

In [152]:
link_list[59]

'https://api.elsevier.com/content/search/sciencedirect?start=5900&count=100&query=hydrogen+production+using+alcohol'

## XML Parsing

In [153]:
# Call the XML_DOI function

for j in range(0, 60):
    
    xmlfile = XML_DOI(link_list[j])
    
    #Read the data
    #<?xml version="1.0" encoding="UTF-8"?>
    tree = ET.parse(r'C:\Users\swath\doi.xml')
    root = tree.getroot()
    #root = ET.fromstring(r'C:\Users\swath\Downloads\test.xml')
    
    #CODE TO EXTRACT DOI

    #Element.findall() finds only elements with a tag which are direct children of the current element
    #Element.find() finds the first child with a particular tag

    #This is the typical xml format for Elsiever
    #-<entry>
    #    <prism:url>https://api.elsevier.com/content/abstract/scopus_id/85111702398</prism:url>
    #    <prism:doi>10.1016/j.jes.2021.07.015</prism:doi>
    #    <dc:description>Aluminosilicate clay mineral (ACM) is a kind of typical raw materials that used widely in manufacturing industry owing to the abundant reserve and low-cost exploring. In past two decades, in-depth understanding on unique layered structure and abundant surface properties endows ACM in the emerging research and application fields. In field of solar-chemical energy conversion, ACM has been widely used to support various semiconductor photocatalysts, forming the composites and achieving efficient conversion of reactants under sunlight irradiation. To date, classic ACM such as kaolinite and montmorillonite, loaded with semiconductor photocatalysts has been widely applied in photocatalysis. This review summaries the recent works on ACM-based composites in photocatalysis. Focusing on the properties of surface and layered structure, we elucidate the different features in the composition with various functional photocatalysts on two typical kinds of ACM, i.e., type 1:1 and type 2:1. Not only large surface area and active surface hydroxyl group assist the substrate adsorption, but also the layered structure provides more space to enlarge the application of ACM-based photocatalysts. Besides, we overview the modifications on ACM from both external surface and the inter-layer space that make the formation of composites more efficiently and boost the photo-chemical process. This review could inspire more upcoming design and synthesis for ACM-based photocatalysts, leading this kind of economic and eco-friendly materials for more practical application in the future.</dc:description>
    #</entry>


    for entry in root.findall('{http://www.w3.org/2005/Atom}entry'):
    
        info_dict = {} #This will create a dictionary where we will store information about the searches
    
        '''
        CHECK CODE
        url = entry.find('{http://prismstandard.org/namespaces/basic/2.0/}url').text
        title = entry.find('{http://purl.org/dc/elements/1.1/}title').text
        pub_name = entry.find('{http://prismstandard.org/namespaces/basic/2.0/}publicationName').text
        doi = entry.find('{http://prismstandard.org/namespaces/basic/2.0/}doi').text
        #description = entry.find('{http://purl.org/dc/elements/1.1/}description').text
        print(url, title, pub_name, doi)
        print('\n')
        '''
    
        info_dict['URL'] = entry.find('{http://prismstandard.org/namespaces/basic/2.0/}url').text
        info_dict['Title'] = entry.find('{http://purl.org/dc/elements/1.1/}title').text
        info_dict['Pub_Name'] = entry.find('{http://prismstandard.org/namespaces/basic/2.0/}publicationName').text
        doi = entry.find('{http://prismstandard.org/namespaces/basic/2.0/}doi')
        if doi is None:
            info_dict['DOI'] = None
        else:
            info_dict['DOI'] = doi.text
    
        info_list.append(info_dict)
    
    #print(info_list)

In [155]:
len(info_list)

6000

## Saving to CSV file

In [156]:
filename = 'doi_6000.csv'
fields = ['URL', 'Title', 'Pub_Name', 'DOI']

with open(filename, 'w', newline='', encoding='utf-8') as f:
    w = csv.DictWriter(f, fieldnames = fields)
    w.writeheader()
    
    for a in info_list:
        w.writerow(a)

In [141]:
#CODE TO EXTRACT DOI

#Element.findall() finds only elements with a tag which are direct children of the current element
#Element.find() finds the first child with a particular tag

#This is the typical xml format for Elsiever
#-<entry>
#    <prism:url>https://api.elsevier.com/content/abstract/scopus_id/85111702398</prism:url>
#    <prism:doi>10.1016/j.jes.2021.07.015</prism:doi>
#    <dc:description>Aluminosilicate clay mineral (ACM) is a kind of typical raw materials that used widely in manufacturing industry owing to the abundant reserve and low-cost exploring. In past two decades, in-depth understanding on unique layered structure and abundant surface properties endows ACM in the emerging research and application fields. In field of solar-chemical energy conversion, ACM has been widely used to support various semiconductor photocatalysts, forming the composites and achieving efficient conversion of reactants under sunlight irradiation. To date, classic ACM such as kaolinite and montmorillonite, loaded with semiconductor photocatalysts has been widely applied in photocatalysis. This review summaries the recent works on ACM-based composites in photocatalysis. Focusing on the properties of surface and layered structure, we elucidate the different features in the composition with various functional photocatalysts on two typical kinds of ACM, i.e., type 1:1 and type 2:1. Not only large surface area and active surface hydroxyl group assist the substrate adsorption, but also the layered structure provides more space to enlarge the application of ACM-based photocatalysts. Besides, we overview the modifications on ACM from both external surface and the inter-layer space that make the formation of composites more efficiently and boost the photo-chemical process. This review could inspire more upcoming design and synthesis for ACM-based photocatalysts, leading this kind of economic and eco-friendly materials for more practical application in the future.</dc:description>
#</entry>


#Manually entered XML with Namespaces
for entry in root.findall('{http://www.w3.org/2005/Atom}entry'):
    
    info_dict = {} #This will create a dictionary where we will store information about the searches
    
    '''
    CHECK CODE
    url = entry.find('{http://prismstandard.org/namespaces/basic/2.0/}url').text
    title = entry.find('{http://purl.org/dc/elements/1.1/}title').text
    pub_name = entry.find('{http://prismstandard.org/namespaces/basic/2.0/}publicationName').text
    doi = entry.find('{http://prismstandard.org/namespaces/basic/2.0/}doi').text
    #description = entry.find('{http://purl.org/dc/elements/1.1/}description').text
    print(url, title, pub_name, doi)
    print('\n')
    '''
    
    info_dict['URL'] = entry.find('{http://prismstandard.org/namespaces/basic/2.0/}url').text
    info_dict['Title'] = entry.find('{http://purl.org/dc/elements/1.1/}title').text
    info_dict['Pub_Name'] = entry.find('{http://prismstandard.org/namespaces/basic/2.0/}publicationName').text
    info_dict['DOI'] = entry.find('{http://prismstandard.org/namespaces/basic/2.0/}doi').text
    
    info_list.append(info_dict)
    
print(info_list)

[{'URL': 'https://api.elsevier.com/content/article/pii/S0040402021007523', 'Title': 'Homogeneous first-row transition metal catalyst for sustainable hydrogen production and organic transformation from methanol, formic acid, and bio-alcohols', 'Pub_Name': 'Tetrahedron', 'DOI': '10.1016/j.tet.2021.132473'}, {'URL': 'https://api.elsevier.com/content/article/pii/S0926337320312856', 'Title': 'Selective photoelectrocatalytic tuning of benzyl alcohol to benzaldehyde for enhanced hydrogen production', 'Pub_Name': 'Applied Catalysis B: Environmental', 'DOI': '10.1016/j.apcatb.2020.119868'}, {'URL': 'https://api.elsevier.com/content/article/pii/S136403212030808X', 'Title': 'Light alcohols reforming towards renewable hydrogen production on multicomponent catalysts', 'Pub_Name': 'Renewable and Sustainable Energy Reviews', 'DOI': '10.1016/j.rser.2020.110523'}, {'URL': 'https://api.elsevier.com/content/article/pii/S016943322101518X', 'Title': 'Minimized Pt deposition on CdS simultaneously maximizes 